# Ações

In [1]:
import requests
from lxml import html
import time
import pandas as pd

# Caminho do CSV com os tickers
caminho_arquivo = r"C:\Users\AGFAKZZ\Desktop\Status Invest Web Scrapping\httpswww.dadosdemercado.com.bracoes.csv" # extraído do site https://www.dadosdemercado.com.br/acoes
df_csv = pd.read_csv(caminho_arquivo)

# Ajuste o nome da coluna de acordo com o conteúdo do seu CSV
lista_acoes = df_csv['Ticker'].dropna().unique().tolist() # extraído do site https://www.dadosdemercado.com.br/acoes
#lista_acoes = ["petr4", "vale3"]

# Lista para armazenar os dados
dados_extraidos = []

def extrair_dados_acao(codigo_acao):
    url = f"https://statusinvest.com.br/acoes/{codigo_acao.lower()}"
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            tree = html.fromstring(response.content)

            def extrair(xpath_expr):
                result = tree.xpath(xpath_expr)
                return result[0].strip() if result else "Não encontrado"

            dados = {
                "Ticker": codigo_acao.upper(),
                "Empresa": extrair('//h1[@title][small]/small/text()'),
                "Valor Atual": extrair('//div[@title="Valor atual do ativo"]/strong/text()'),
                "Variação Dia": extrair('//span[@title="Variação do valor do ativo com base no dia anterior"]/b/text()'),
                "Mín 52 Semanas": extrair('//div[@title="Valor mínimo das últimas 52 semanas"]/strong/text()'),
                "Mín Mês": extrair('//div[@title="Valor mínimo do mês atual"]/span[@class="sub-value"]/text()'),
                "Máx 52 Semanas": extrair('//div[@title="Valor máximo das últimas 52 semanas"]/strong/text()'),
                "Máx Mês": extrair('//div[@title="Valor máximo do mês atual"]/span[@class="sub-value"]/text()'),
                "Dividend Yield": extrair('//div[@title="Dividend Yield com base nos últimos 12 meses"]/strong/text()'),
                "Dividend Yield (soma)": extrair('//div[@title="Soma total de proventos distribuídos nos últimos 12 meses"]/span[@class="sub-value"]/text()'),
                "Valorização 12M": extrair('//div[@title="Valorização no preço do ativo com base nos últimos 12 meses"]/strong/text()'),
                "Valorização Mês": extrair('//div[@title="Valorização no preço do ativo com base no mês atual"]//b/text()'),
                "P/L": extrair("//h3[text()='P/L']/ancestor::div[contains(@title, 'mercado está disposto')]/descendant::strong[contains(@class, 'value')]/text()"),
                "PEG Ratio": extrair("//h3[text()='PEG Ratio']/following::strong[contains(@class, 'value')][1]/text()"),
                "P/VP": extrair("//h3[text()='P/VP']/ancestor::div[contains(@title, 'Facilita a análise')]/descendant::strong[contains(@class, 'value')]/text()"),
                "EV/EBITDA": extrair("//h3[text()='EV/EBITDA']/ancestor::div[contains(@title, 'O EV (Enterprise')]/descendant::strong[contains(@class, 'value')]/text()"),
                "EV/EBIT": extrair("//h3[text()='EV/EBIT']/ancestor::div[contains(@title, 'O EV (Enterprise')]/descendant::strong[contains(@class, 'value')]/text()"),
                "P/EBITDA": extrair("//h3[text()='P/EBITDA']/ancestor::div[contains(@title, 'O EBITDA permite')]/descendant::strong[contains(@class, 'value')]/text()"),
                "P/EBIT": extrair("//h3[text()='P/EBIT']/ancestor::div[contains(@title, 'Indica qual')]/descendant::strong[contains(@class, 'value')]/text()"),
                "VPA": extrair("//h3[text()='VPA']/ancestor::div[contains(@title, 'Indica qual')]/descendant::strong[contains(@class, 'value')]/text()"),
                "P/ATIVO": extrair("//h3[text()='P/Ativo']/ancestor::div[contains(@title, 'Preço da ação')]/descendant::strong[contains(@class, 'value')]/text()"),
                "LPA": extrair("//h3[text()='LPA']/ancestor::div[contains(@title, 'Indicar se a empresa')]/descendant::strong[contains(@class, 'value')]/text()"),
                "P/SR": extrair("//h3[text()='P/SR']/ancestor::div[contains(@title, 'Valor de mercado da empresa')]/descendant::strong[contains(@class, 'value')]/text()"),
                "P/Cap. Giro": extrair("//h3[text()='P/Cap. Giro']/ancestor::div[contains(@title, 'Preço da ação')]/descendant::strong[contains(@class, 'value')]/text()"),
                "P/Ativo Circ. Liq.": extrair("//h3[text()='P/Ativo Circ. Liq.']/ancestor::div[contains(@title, 'É a diferença entre o ativo')]/descendant::strong[contains(@class, 'value')]/text()"),
                "Dív. líquida/PL": extrair("//h3[text()='Dív. líquida/PL']/ancestor::div[contains(@title, 'Indica quanto')]/descendant::strong[contains(@class, 'value')]/text()"),
                "Dív. líquida/EBITDA": extrair("//h3[text()='Dív. líquida/EBITDA']/ancestor::div[contains(@title, 'Indica')]/descendant::strong[contains(@class, 'value')]/text()"),
                "Dív. líquida/EBIT": extrair("//h3[text()='Dív. líquida/EBIT']/ancestor::div[contains(@title, 'Indica')]/descendant::strong[contains(@class, 'value')]/text()"),
                "PL/Ativos": extrair("//h3[text()='PL/Ativos']/ancestor::div[contains(@title, 'O Patrimônio')]/descendant::strong[contains(@class, 'value')]/text()"),
                "Passivos/Ativos": extrair("//h3[text()='Passivos/Ativos']/ancestor::div[contains(@title, 'Calculo')]/descendant::strong[contains(@class, 'value')]/text()"),
                "Liq. corrente": extrair("//h3[text()='Liq. corrente']/ancestor::div[contains(@title, 'Indica')]/descendant::strong[contains(@class, 'value')]/text()"),
                "M. Bruta": extrair("//h3[text()='M. Bruta']/ancestor::div[contains(@title, 'Mede')]/descendant::strong[contains(@class, 'value')]/text()"),
                "M. EBITDA": extrair("//h3[text()='M. EBITDA']/ancestor::div[contains(@title, 'É o percentual')]/descendant::strong[contains(@class, 'value')]/text()"),
                "M. EBIT": extrair("//h3[text()='M. EBIT']/ancestor::div[contains(@title, 'Útil')]/descendant::strong[contains(@class, 'value')]/text()"),
                "M. Líquida": extrair("//h3[text()='M. Líquida']/ancestor::div[contains(@title, 'Revela')]/descendant::strong[contains(@class, 'value')]/text()"),
                "ROE": extrair("//h3[text()='ROE']/ancestor::div[contains(@title, 'Mede')]/descendant::strong[contains(@class, 'value')]/text()"),
                "ROA": extrair("//h3[text()='ROA']/ancestor::div[contains(@title, 'O retorno')]/descendant::strong[contains(@class, 'value')]/text()"),
                "ROIC": extrair("//h3[text()='ROIC']/ancestor::div[contains(@title, 'Mede')]/descendant::strong[contains(@class, 'value')]/text()"),
                "Giro ativos": extrair("//h3[text()='Giro ativos']/ancestor::div[contains(@title, 'Mede')]/descendant::strong[contains(@class, 'value')]/text()"),
                "CAGR Receitas 5 anos": extrair("//h3[text()='CAGR Receitas 5 anos']/ancestor::div[contains(@title, 'O CAGR')]/descendant::strong[contains(@class, 'value')]/text()"),
                "CAGR Lucros 5 anos": extrair("//h3[text()='CAGR Lucros 5 anos']/ancestor::div[contains(@title, 'O CAGR')]/descendant::strong[contains(@class, 'value')]/text()"),
                "Tipo": extrair("//h3[text()='Tipo']/following-sibling::strong[@class='value']/text()"),
                "Tag Along": extrair("//span[contains(., 'Tag Along')]/ancestor::div[2]//strong[@class='value']/text()"),
                "Liq. méd. diária": extrair("//span[contains(., 'Liquidez média diária') or contains(., 'Liq. méd. diária')]/ancestor::div[2]//strong[@class='value']/text()"),
                "Patrimônio líquido": extrair("//h3[text()='Patrimônio líquido']/ancestor::div[contains(@title, 'É uma')]/descendant::strong[contains(@class, 'value')]/text()"),
                "Ativos": extrair("//h3[text()='Ativos']/ancestor::div[contains(@title, 'Ativo')]/descendant::strong[contains(@class, 'value')]/text()"),
                "Ativo circulante": extrair("//h3[text()='Ativo circulante']/ancestor::div[contains(@title, 'Ativo')]/descendant::strong[contains(@class, 'value')]/text()"),
                "Dívida bruta": extrair("//h3[text()='Dívida bruta']/ancestor::div[contains(@title, 'Representa')]/descendant::strong[contains(@class, 'value')]/text()"),
                "Disponibilidade": extrair("//h3[text()='Disponibilidade']/ancestor::div[contains(@title, 'Representa')]/descendant::strong[contains(@class, 'value')]/text()"),
                "Dívida líquida": extrair("//h3[text()='Dívida líquida']/ancestor::div[contains(@title, 'A dívida')]/descendant::strong[contains(@class, 'value')]/text()"),
                "Valor de mercado": extrair("//h3[text()='Valor de mercado']/ancestor::div[contains(@title, 'O valor')]/descendant::strong[contains(@class, 'value')]/text()"),
                "Valor de firma": extrair("//h3[text()='Valor de firma']/ancestor::div[contains(@title, 'Soma do valor')]/descendant::strong[contains(@class, 'value')]/text()"),
                "Nº total de papéis": extrair("//h3[span[contains(text(), 'Nº total de papéis')]]/following-sibling::strong/text()"),
                "Segmento de listagem": extrair("//h3[text()='Segmento de listagem']/ancestor::div[contains(@title, 'Segmento')]/descendant::strong[contains(@class, 'value')]/text()"),
                "Free Float": extrair("//h3[text()='Free Float']/../../following-sibling::strong[@class='value']/text()"),
                "Setor de Atuação": extrair("//span[contains(text(), 'Setor de Atuação')]/following::strong[@class='value'][1]/text()"),
                "Subsetor de Atuação": extrair("//span[contains(text(), 'Subsetor de Atuação')]/following::strong[@class='value'][1]/text()"),
                "Segmento de Atuação": extrair("//span[contains(text(), 'Segmento de Atuação')]/following::strong[@class='value'][1]/text()")
            }

            dados_extraidos.append(dados)
        else:
            print(f"Erro ao acessar {codigo_acao}: {response.status_code}")
    except Exception as e:
        print(f"Erro ao buscar {codigo_acao}: {e}")

# Processar os tickers
for acao in lista_acoes:
    extrair_dados_acao(acao)
    time.sleep(1)  # Evita sobrecarga

# Converter para DataFrame
df_acoes = pd.DataFrame(dados_extraidos)

# Exibir os resultados
df_acoes

# Salvar o DataFrame em CSV
caminho_arquivo = r"C:\Users\AGFAKZZ\Desktop\Status Invest Web Scrapping\df_acoes.csv"
df_acoes.to_csv(caminho_arquivo, index=False, encoding='utf-8-sig')

Erro ao acessar DASA11: 404
Erro ao acessar AMAR11: 404
Erro ao acessar ATED3: 404
Erro ao acessar BIED3: 404
Erro ao acessar AZEV11: 404
Erro ao acessar EQMA3B: 404
Erro ao acessar PSVM11: 404
Erro ao acessar PINE11: 404


# FIIs

In [2]:
import requests
from lxml import html
import time
import pandas as pd

pd.set_option('display.max_columns', None)

# Caminho do CSV com os tickers
caminho_arquivo = r"C:\Users\AGFAKZZ\Desktop\Status Invest Web Scrapping\fundosListados.csv" # extraído do site lista_fundos_imobiliarios
df_csv = pd.read_csv(caminho_arquivo, encoding="latin1", sep=";")

# Ajuste o nome da coluna de acordo com o conteúdo do seu CSV
lista_fundos_imobiliarios = df_csv['Segmento'].dropna().unique().tolist() # extraído do site https://www.b3.com.br/pt_br/produtos-e-servicos/negociacao/renda-variavel/fundos-de-investimentos/fii/fiis-listados/
lista_fundos_imobiliarios = [str(item) if str(item).endswith("11") else str(item) + "11" for item in lista_fundos_imobiliarios]
#lista_fundos_imobiliarios = ["mxrf11", "irdm11"]

# Lista para armazenar os dados
dados_extraidos = []

def extrair_dados_acao(codigo_fundo_imobiliario):
    url = f"https://statusinvest.com.br/fundos-imobiliarios/{codigo_fundo_imobiliario.lower()}"
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            tree = html.fromstring(response.content)

            def extrair(xpath_expr):
                result = tree.xpath(xpath_expr)
                return result[0].strip() if result else "Não encontrado"

            dados = {
                "Ticker": codigo_fundo_imobiliario.upper(),
                "Empresa": extrair("//h1[@class='lh-4']/small/text()"),
                "Valor Atual": extrair('//div[@title="Valor atual do ativo"]/strong/text()'),
                "Variação Dia": extrair('//span[@title="Variação do valor do ativo com base no dia anterior"]/b/text()'),
                "Mín 52 Semanas": extrair('//div[@title="Valor mínimo das últimas 52 semanas"]/strong/text()'),
                "Mín Mês": extrair('//div[@title="Valor mínimo do mês atual"]/span[@class="sub-value"]/text()'),
                "Máx 52 Semanas": extrair('//div[@title="Valor máximo das últimas 52 semanas"]/strong/text()'),
                "Máx Mês": extrair('//div[@title="Valor máximo do mês atual"]/span[@class="sub-value"]/text()'),
                "Dividend Yield": extrair('//div[@title="Dividend Yield com base nos últimos 12 meses"]/strong/text()'),
                "Dividend Yield (soma)": extrair('//div[@title="Soma total de proventos distribuídos nos últimos 12 meses"]/span[@class="sub-value"]/text()'),
                "Valorização 12M": extrair('//div[@title="Valorização no preço do ativo com base nos últimos 12 meses"]/strong/text()'),
                "Valor patrim. p/cota": extrair("//span[@class='title d-none d-sm-block d-lg-none' and normalize-space(text())='Valor patrim. p/cota']/following-sibling::strong[@class='value']/text()"),
                "Patrimônio": extrair("//span[@class='sub-title' and normalize-space(text())='Patrimônio']/following-sibling::span[@class='sub-value']/text()"),
                "P/VP": extrair("//h3[@class='title m-0' and normalize-space(text())='P/VP']/following-sibling::strong[@class='value']/text()"),
                "Valor de Mercado": extrair("//span[@class='sub-title' and normalize-space(text())='Valor de mercado']/following-sibling::span[@class='sub-value']/text()"),
                "Valor em Caixa %": extrair("//h3[@class='title m-0' and normalize-space(text())='Valor em caixa']/following-sibling::strong[@class='value']/text()"),
                "Total em Caixa": extrair("//span[@class='sub-title' and normalize-space(text())='Total']/following-sibling::span[@class='sub-value']/text()"),
                "Número de Cotistas": extrair("//h3[@class='title m-0' and normalize-space(text())='Nº de Cotistas']/following-sibling::strong[@class='value']/text()"),
                "Número de Cotas": extrair("//span[@class='sub-title' and normalize-space(text())='Nº de Cotas']/following-sibling::span[@class='sub-value']/text()"),
                "Rendimento Médio (24M)": extrair("//span[@class='title d-none d-xs-block d-xl-none' and normalize-space(text())='RENDIM. MÉDIO (24M)']/following-sibling::strong[@class='value']/text()"),
                "Taxa de Administração": extrair("//span[@class='d-none d-sm-inline-block' and contains(normalize-space(text()), 'Taxas Administração')]/following::strong[@class='value'][1]/text()"),
                "Liquidez Média Diária": extrair("//span[@class='d-none d-lg-inline-block' and normalize-space(text())='Liquidez média diária']/following::strong[@class='value'][1]/text()"),
                "Participação no IFIX": extrair("//span[@class='title d-block d-sm-none' and normalize-space(text())='PARTIC. NO IFIX']/following::strong[@class='value'][1]/text()"),
                "Próximo Rendimento": extrair("//div[@class='bg-secondary white-text card w-100 w-md-45']//div[@class='d-flex align-items-center']//strong[contains(@class, 'value')]/text()"),
                "Próximo Rendimento %": extrair("//div[@class='bg-secondary white-text card w-100 w-md-45']//small[normalize-space(text())='Rendimento']/following::b[@class='sub-value fs-4 lh-3'][1]/text()"),
                "Próximo Rendimento Cotação Base": extrair("//div[@class='bg-secondary white-text card w-100 w-md-45'][.//small[normalize-space(text())='Cotação base']]//b[contains(@class, 'sub-value')]/text()"),
                "Segmento": extrair("//div[contains(@class, 'top-info')]/span[text()='Tipo de gestão']/following-sibling::span[@class='value']")
            }

            dados_extraidos.append(dados)
        else:
            print(f"Erro ao acessar {codigo_fundo_imobiliario}: {response.status_code}")
    except Exception as e:
        print(f"Erro ao buscar {codigo_fundo_imobiliario}: {e}")

# Processar os tickers
for fundo_imobiliario in lista_fundos_imobiliarios:
    extrair_dados_acao(fundo_imobiliario)
    time.sleep(1)  # Evita sobrecarga

# Converter para DataFrame
df_fiis = pd.DataFrame(dados_extraidos)

# Exibir os resultados
df_fiis

# Salvar o DataFrame em CSV
caminho_arquivo = r"C:\Users\AGFAKZZ\Desktop\Status Invest Web Scrapping\df_fiis.csv"
df_fiis.to_csv(caminho_arquivo, index=False, encoding='utf-8-sig')

# Stocks

In [3]:
import pandas as pd
import requests
from lxml import html
import time

# Parte 1 – Obter os tickers do S&P 500 a partir do site stockanalysis.com
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'
}
url = 'https://stockanalysis.com/list/sp-500-stocks/'
response = requests.get(url, headers=headers)
tables = pd.read_html(response.text)
df = tables[0]

# Lista de tickers a partir da coluna Symbol
lista_acoes = df['Symbol'].dropna().unique().tolist()

# Parte 2 – Extrair os dados do site StatusInvest
dados_extraidos = []

def extrair_dados_acao(codigo_acao):
    url = f"https://statusinvest.com.br/acoes/eua/{codigo_acao.lower()}"
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            tree = html.fromstring(response.content)

            def extrair(xpath_expr):
                result = tree.xpath(xpath_expr)
                return result[0].strip() if result else "Não encontrado"

            dados = {
                "Ticker": codigo_acao.upper(),
                "Empresa": extrair('//h1[@title][small]/small/text()'),
                "Valor Atual": extrair('//div[@title="Valor atual do ativo"]/strong/text()'),
                "Variação Dia": extrair('//span[@title="Variação do valor do ativo com base no dia anterior"]/b/text()'),
                "Mín 52 Semanas": extrair('//div[@title="Valor mínimo das últimas 52 semanas"]/strong/text()'),
                "Mín Mês": extrair('//div[@title="Valor mínimo do mês atual"]/span[@class="sub-value"]/text()'),
                "Máx 52 Semanas": extrair('//div[@title="Valor máximo das últimas 52 semanas"]/strong/text()'),
                "Máx Mês": extrair('//div[@title="Valor máximo do mês atual"]/span[@class="sub-value"]/text()'),
                "Dividend Yield": extrair('//div[@title="Dividend Yield com base nos últimos 12 meses"]/strong/text()'),
                "Dividend Yield (soma)": extrair('//div[@title="Soma total de proventos distribuídos nos últimos 12 meses"]/span[@class="sub-value"]/text()'),
                "Valorização 12M": extrair('//div[@title="Valorização no preço do ativo com base nos últimos 12 meses"]/strong/text()'),
                "Valorização Mês": extrair('//div[@title="Valorização no preço do ativo com base no mês atual"]//b/text()'),
                "P/L": extrair("//h3[text()='P/L']/ancestor::div[contains(@title, 'mercado está disposto')]/descendant::strong[contains(@class, 'value')]/text()"),
                "PEG Ratio": extrair("//h3[text()='PEG Ratio']/following::strong[contains(@class, 'value')][1]/text()"),
                "P/VP": extrair("//h3[text()='P/VP']/ancestor::div[contains(@title, 'Facilita a análise')]/descendant::strong[contains(@class, 'value')]/text()"),
                "EV/EBITDA": extrair("//h3[text()='EV/EBITDA']/ancestor::div[contains(@title, 'O EV (Enterprise')]/descendant::strong[contains(@class, 'value')]/text()"),
                "EV/EBIT": extrair("//h3[text()='EV/EBIT']/ancestor::div[contains(@title, 'O EV (Enterprise')]/descendant::strong[contains(@class, 'value')]/text()"),
                "P/EBITDA": extrair("//h3[text()='P/EBITDA']/ancestor::div[contains(@title, 'O EBITDA permite')]/descendant::strong[contains(@class, 'value')]/text()"),
                "P/EBIT": extrair("//h3[text()='P/EBIT']/ancestor::div[contains(@title, 'Indica qual')]/descendant::strong[contains(@class, 'value')]/text()"),
                "VPA": extrair("//h3[text()='VPA']/ancestor::div[contains(@title, 'Indica qual')]/descendant::strong[contains(@class, 'value')]/text()"),
                "P/ATIVO": extrair("//h3[text()='P/Ativo']/ancestor::div[contains(@title, 'Preço da ação')]/descendant::strong[contains(@class, 'value')]/text()"),
                "LPA": extrair("//h3[text()='LPA']/ancestor::div[contains(@title, 'Indicar se a empresa')]/descendant::strong[contains(@class, 'value')]/text()"),
                "P/SR": extrair("//h3[text()='P/SR']/ancestor::div[contains(@title, 'Valor de mercado da empresa')]/descendant::strong[contains(@class, 'value')]/text()"),
                "P/Cap. Giro": extrair("//h3[text()='P/Cap. Giro']/ancestor::div[contains(@title, 'Preço da ação')]/descendant::strong[contains(@class, 'value')]/text()"),
                "P/Ativo Circ. Liq.": extrair("//h3[text()='P/Ativo Circ. Liq.']/ancestor::div[contains(@title, 'É a diferença entre o ativo')]/descendant::strong[contains(@class, 'value')]/text()"),
                "Dív. líquida/PL": extrair("//h3[text()='Dív. líquida/PL']/ancestor::div[contains(@title, 'Indica quanto')]/descendant::strong[contains(@class, 'value')]/text()"),
                "Dív. líquida/EBITDA": extrair("//h3[text()='Dív. líquida/EBITDA']/ancestor::div[contains(@title, 'Indica')]/descendant::strong[contains(@class, 'value')]/text()"),
                "Dív. líquida/EBIT": extrair("//h3[text()='Dív. líquida/EBIT']/ancestor::div[contains(@title, 'Indica')]/descendant::strong[contains(@class, 'value')]/text()"),
                "PL/Ativos": extrair("//h3[text()='PL/Ativos']/ancestor::div[contains(@title, 'O Patrimônio')]/descendant::strong[contains(@class, 'value')]/text()"),
                "Passivos/Ativos": extrair("//h3[text()='Passivos/Ativos']/ancestor::div[contains(@title, 'Calculo')]/descendant::strong[contains(@class, 'value')]/text()"),
                "Liq. corrente": extrair("//h3[text()='Liq. corrente']/ancestor::div[contains(@title, 'Indica')]/descendant::strong[contains(@class, 'value')]/text()"),
                "M. Bruta": extrair("//h3[text()='M. Bruta']/ancestor::div[contains(@title, 'Mede')]/descendant::strong[contains(@class, 'value')]/text()"),
                "M. EBITDA": extrair("//h3[text()='M. EBITDA']/ancestor::div[contains(@title, 'É o percentual')]/descendant::strong[contains(@class, 'value')]/text()"),
                "M. EBIT": extrair("//h3[text()='M. EBIT']/ancestor::div[contains(@title, 'Útil')]/descendant::strong[contains(@class, 'value')]/text()"),
                "M. Líquida": extrair("//h3[text()='M. Líquida']/ancestor::div[contains(@title, 'Revela')]/descendant::strong[contains(@class, 'value')]/text()"),
                "ROE": extrair("//h3[text()='ROE']/ancestor::div[contains(@title, 'Mede')]/descendant::strong[contains(@class, 'value')]/text()"),
                "ROA": extrair("//h3[text()='ROA']/ancestor::div[contains(@title, 'O retorno')]/descendant::strong[contains(@class, 'value')]/text()"),
                "ROIC": extrair("//h3[text()='ROIC']/ancestor::div[contains(@title, 'Mede')]/descendant::strong[contains(@class, 'value')]/text()"),
                "Giro ativos": extrair("//h3[text()='Giro ativos']/ancestor::div[contains(@title, 'Mede')]/descendant::strong[contains(@class, 'value')]/text()"),
                "CAGR Receitas 5 anos": extrair("//h3[text()='CAGR Receitas 5 anos']/ancestor::div[contains(@title, 'O CAGR')]/descendant::strong[contains(@class, 'value')]/text()"),
                "CAGR Lucros 5 anos": extrair("//h3[text()='CAGR Lucros 5 anos']/ancestor::div[contains(@title, 'O CAGR')]/descendant::strong[contains(@class, 'value')]/text()"),
                "Patrimônio líquido": extrair("//h3[text()='Patrimônio líquido']/ancestor::div[contains(@title, 'É uma')]/descendant::strong[contains(@class, 'value')]/text()"),
                "Ativos": extrair("//h3[text()='Ativos']/ancestor::div[contains(@title, 'Ativo')]/descendant::strong[contains(@class, 'value')]/text()"),
                "Ativo circulante": extrair("//h3[text()='Ativo circulante']/ancestor::div[contains(@title, 'Ativo')]/descendant::strong[contains(@class, 'value')]/text()"),
                "Dívida bruta": extrair("//h3[text()='Dívida bruta']/ancestor::div[contains(@title, 'Representa')]/descendant::strong[contains(@class, 'value')]/text()"),
                "Disponibilidade": extrair("//h3[text()='Disponibilidade']/ancestor::div[contains(@title, 'Representa')]/descendant::strong[contains(@class, 'value')]/text()"),
                "Dívida líquida": extrair("//h3[text()='Dívida líquida']/ancestor::div[contains(@title, 'A dívida')]/descendant::strong[contains(@class, 'value')]/text()"),
                "Valor de mercado": extrair("//h3[text()='Valor de mercado']/ancestor::div[contains(@title, 'O valor')]/descendant::strong[contains(@class, 'value')]/text()"),
                "Valor de firma": extrair("//h3[text()='Valor de firma']/ancestor::div[contains(@title, 'Soma do valor')]/descendant::strong[contains(@class, 'value')]/text()"),
                "Setor": extrair("//div[@class='info pl-md-2 pr-md-2 w-md-33'][.//span[@class='sub-value' and normalize-space(text())='SETOR']]//strong[@class='value']/text()"),
                "Indústria": extrair("//div[@class='info pl-md-2 pr-md-2 w-md-65'][.//span[@class='sub-value' and normalize-space(text())='INDÚSTRIA']]//strong[@class='value']/text()")
            }

            dados_extraidos.append(dados)
        else:
            print(f"Erro ao acessar {codigo_acao}: {response.status_code}")
    except Exception as e:
        print(f"Erro ao buscar {codigo_acao}: {e}")

# Processar os tickers (aqui limitado aos 10 primeiros para evitar sobrecarga durante testes)
for acao in lista_acoes:
    extrair_dados_acao(acao)
    time.sleep(1)  # Delay para não sobrecarregar o servidor

# Converter os dados em DataFrame
df_stocks = pd.DataFrame(dados_extraidos)

# Exibir os resultados
df_stocks

# Salvar o DataFrame em CSV
caminho_arquivo = r"C:\Users\AGFAKZZ\Desktop\Status Invest Web Scrapping\df_stocks.csv"
df_stocks.to_csv(caminho_arquivo, index=False, encoding='utf-8-sig')

C:\Users\AGFAKZZ\AppData\Local\Temp\ipykernel_13548\3138903207.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


# Índices de ações

In [4]:
import requests
from lxml import html
import time
import pandas as pd

pd.set_option('display.max_columns', None)

lista_indices_acoes = [
    "ibovespa",
    "indice-dividendos",
    "indice-small-cap",
    "indice-brasil-amplo",
    "indice-brasil-100",
    "indice-brasil-50",
    "indice-carbono-eficiente",
    "indice-de-consumo",
    "indice-de-energia-eletrica",
    "indice-financeiro",
    "indice-de-governanca-corporativa-trade",
    "indice-de-acoes-com-governanca-corporativa-diferenciada",
    "indice-de-governanca-corporativa-novo-mercado",
    "indice-de-materiais-basicos",
    "indice-imobiliario",
    "indice-do-setor-industrial",
    "indice-de-sustentabilidade-empresarial",
    "indice-de-acoes-com-tag-along-diferenciado",
    "indice-valor",
    "indice-midlarge-cap",
    "indice-utilidade-publica"
]

# Lista para armazenar os dados
dados_extraidos = []

def extrair_dados_acao(codigo_indices_acoes):
    url = f"https://statusinvest.com.br/indices/{codigo_indices_acoes.lower()}"
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            tree = html.fromstring(response.content)

            def extrair(xpath_expr):
                result = tree.xpath(xpath_expr)
                return result[0].strip() if result else ""

            dados = {
                "Ticker": codigo_indices_acoes.replace("-", " ").title(),
                "Valor Atual": extrair('//div[@title="Valor (teórico) atual do índice"]//strong[@class="value"]/text()'),
                "Variação": extrair('//div[@title="Variação do valor do índice"]//strong[@class="value"]/text()'),
                "Mín 52 Semanas": extrair('//div[@title="Valor mínimo das últimas 52 semanas"]/strong/text()'),
                "Mín Mês": extrair("//div[@class='d-flex justify-between' and @title='Valor mínimo do mês atual']//span[@class='sub-value']/text()"),
                "Máx 52 Semanas": extrair('//div[@title="Valor máximo das últimas 52 semanas"]/strong/text()'),
                "Máx Mês": extrair('//div[@title="Valor máximo do mês atual"]/span[@class="sub-value"]/text()'),
                "DY": extrair("//h3[text()='D.Y']/ancestor::div[contains(@title, 'proventos pagos por uma companhia')]/descendant::strong[contains(@class, 'value')]/text()"),
                "P/L": extrair("//h3[text()='P/L']/ancestor::div[contains(@title, 'mercado está disposto')]/descendant::strong[contains(@class, 'value')]/text()"),
                "P/VP": extrair("//h3[text()='P/VP']/ancestor::div[contains(@title, 'Facilita a análise')]/descendant::strong[contains(@class, 'value')]/text()"),
                "EV/EBIT": extrair("//h3[text()='EV/EBIT']/ancestor::div[contains(@title, 'O EV (Enterprise')]/descendant::strong[contains(@class, 'value')]/text()"),
                "P/EBIT": extrair("//h3[text()='P/EBIT']/ancestor::div[contains(@title, 'Indica qual')]/descendant::strong[contains(@class, 'value')]/text()"),
                "VPA": extrair("//h3[text()='VPA']/ancestor::div[contains(@title, 'Indica qual')]/descendant::strong[contains(@class, 'value')]/text()"),
                "P/ATIVO": extrair("//h3[text()='P/Ativo']/ancestor::div[contains(@title, 'Preço da ação')]/descendant::strong[contains(@class, 'value')]/text()"),
                "LPA": extrair("//h3[text()='LPA']/ancestor::div[contains(@title, 'Indicar se a empresa')]/descendant::strong[contains(@class, 'value')]/text()"),
                "P/SR": extrair("//h3[text()='P/SR']/ancestor::div[contains(@title, 'Valor de mercado da empresa')]/descendant::strong[contains(@class, 'value')]/text()"),
                "M. Bruta": extrair("//h3[text()='M. Bruta']/ancestor::div[contains(@title, 'Mede')]/descendant::strong[contains(@class, 'value')]/text()"),
                "M. EBIT": extrair("//h3[text()='M. EBIT']/ancestor::div[contains(@title, 'Útil')]/descendant::strong[contains(@class, 'value')]/text()"),
                "M. Líquida": extrair("//h3[text()='M. Líquida']/ancestor::div[contains(@title, 'Revela')]/descendant::strong[contains(@class, 'value')]/text()"),
                "ROE": extrair("//h3[text()='ROE']/ancestor::div[contains(@title, 'Mede')]/descendant::strong[contains(@class, 'value')]/text()"),
                "ROA": extrair("//h3[text()='ROA']/ancestor::div[contains(@title, 'O retorno')]/descendant::strong[contains(@class, 'value')]/text()"),
                "ROIC": extrair("//h3[text()='ROIC']/ancestor::div[contains(@title, 'Mede')]/descendant::strong[contains(@class, 'value')]/text()"),
                "CAGR Receitas 5 anos": extrair("//h3[text()='CAGR Receitas 5 anos']/ancestor::div[contains(@title, 'O CAGR')]/descendant::strong[contains(@class, 'value')]/text()"),
                "CAGR Lucros 5 anos": extrair("//h3[text()='CAGR Lucros 5 anos']/ancestor::div[contains(@title, 'O CAGR')]/descendant::strong[contains(@class, 'value')]/text()"),
                
            }

            dados_extraidos.append(dados)
        else:
            print(f"Erro ao acessar {codigo_indices_acoes}: {response.status_code}")
    except Exception as e:
        print(f"Erro ao buscar {codigo_indices_acoes}: {e}")

# Processar os tickers
for indices_acoes in lista_indices_acoes:
    extrair_dados_acao(indices_acoes)
    time.sleep(1)  # Evita sobrecarga

# Converter para DataFrame
df_indices_acoes = pd.DataFrame(dados_extraidos)

# Exibir os resultados
df_indices_acoes

# Salvar o DataFrame em CSV
caminho_arquivo = r"C:\Users\AGFAKZZ\Desktop\Status Invest Web Scrapping\df_indice_acoes.csv"
df_indices_acoes.to_csv(caminho_arquivo, index=False, encoding='utf-8-sig')

# Indexadores

In [5]:
import requests
from lxml import html
import time
import pandas as pd
from datetime import datetime, timedelta

pd.set_option('display.max_columns', None)

lista_indexadores = ["selic", "ipca", "cdi", "igpm"]
dados_extraidos = []

# Dicionário com os nomes dos meses em português
meses_pt = {
    1: "Janeiro", 2: "Fevereiro", 3: "Março", 4: "Abril",
    5: "Maio", 6: "Junho", 7: "Julho", 8: "Agosto",
    9: "Setembro", 10: "Outubro", 11: "Novembro", 12: "Dezembro"
}

# Obter mês anterior
hoje = datetime.today()
primeiro_dia_mes_atual = hoje.replace(day=1)
ultimo_mes = primeiro_dia_mes_atual - timedelta(days=1)
mes_ano_anterior = f"{meses_pt[ultimo_mes.month]}, {ultimo_mes.year}"

def extrair_dados_acao(codigo_indexador):
    url = f"https://statusinvest.com.br/indices/{codigo_indexador.lower()}"
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            tree = html.fromstring(response.content)

            def extrair(xpath_expr):
                result = tree.xpath(xpath_expr)
                return result[0].strip() if result else "Não encontrado"

            dados = {
                "Ticker": codigo_indexador.upper(),
                "Valor Atual": extrair(f"//h2[contains(translate(text(), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), '{codigo_indexador} hoje')]/following-sibling::strong[@class='value']/text()"),
                "Variação Mês": extrair(f"//span[text()='{mes_ano_anterior}']/ancestor::div//span[@class='sub-value']/text()"),
            }

            dados_extraidos.append(dados)
        else:
            print(f"Erro ao acessar {codigo_indexador}: {response.status_code}")
    except Exception as e:
        print(f"Erro ao buscar {codigo_indexador}: {e}")

# Executar
for indexador in lista_indexadores:
    extrair_dados_acao(indexador)
    time.sleep(1)

df_indexador = pd.DataFrame(dados_extraidos)
df_indexador

# Salvar o DataFrame em CSV
caminho_arquivo = r"C:\Users\AGFAKZZ\Desktop\Status Invest Web Scrapping\df_indexadores.csv"
df_indexador.to_csv(caminho_arquivo, index=False, encoding='utf-8-sig')

# Índices de FIIs

In [6]:
import requests
from lxml import html
import time
import pandas as pd

pd.set_option('display.max_columns', None)

lista_indices_FIIs = [
    "ifix",
    "suno-30",
    "indice-xp-de-fundos-imobiliarios",
    "indice-xp-de-fundos-imobiliarios-de-papel",
    "indice-xp-de-fundos-imobiliarios-de-tijolo"
]

# Lista para armazenar os dados
dados_extraidos = []

def extrair_dados_acao(codigo_indices_FIIs):
    url = f"https://statusinvest.com.br/indices/{codigo_indices_FIIs.lower()}"
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            tree = html.fromstring(response.content)

            def extrair(xpath_expr):
                result = tree.xpath(xpath_expr)
                return result[0].strip() if result else ""

            dados = {
                "Ticker": codigo_indices_FIIs.replace("-", " ").title(),
                "Valor Atual": extrair('//div[@title="Valor (teórico) atual do índice"]//strong[@class="value"]/text()'),
                "Variação": extrair('//div[@title="Variação do valor do índice"]//strong[@class="value"]/text()'),
                "Mín 52 Semanas": extrair('//div[@title="Valor mínimo das últimas 52 semanas"]/strong/text()'),
                "Mín Mês": extrair("//div[@class='d-flex justify-between' and @title='Valor mínimo do mês atual']//span[@class='sub-value']/text()"),
                "Máx 52 Semanas": extrair('//div[@title="Valor máximo das últimas 52 semanas"]/strong/text()'),
                "Máx Mês": extrair('//div[@title="Valor máximo do mês atual"]/span[@class="sub-value"]/text()')  
            }

            dados_extraidos.append(dados)
        else:
            print(f"Erro ao acessar {codigo_indices_FIIs}: {response.status_code}")
    except Exception as e:
        print(f"Erro ao buscar {codigo_indices_FIIs}: {e}")

# Processar os tickers
for indices_FIIs in lista_indices_FIIs:
    extrair_dados_acao(indices_FIIs)
    time.sleep(1)  # Evita sobrecarga

# Converter para DataFrame
df_indices_FIIs = pd.DataFrame(dados_extraidos)

# Exibir os resultados
df_indices_FIIs

# Salvar o DataFrame em CSV
caminho_arquivo = r"C:\Users\AGFAKZZ\Desktop\Status Invest Web Scrapping\df_indice_fiis.csv"
df_indices_FIIs.to_csv(caminho_arquivo, index=False, encoding='utf-8-sig')

# Índice de BDRs

In [7]:
import requests
from lxml import html
import time
import pandas as pd

pd.set_option('display.max_columns', None)

lista_indices_BDRs = [
    "indice-de-bdrs-nao-patrocinados-global"
]

# Lista para armazenar os dados
dados_extraidos = []

def extrair_dados_acao(codigo_indices_BDRs):
    url = f"https://statusinvest.com.br/indices/{codigo_indices_BDRs.lower()}"
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            tree = html.fromstring(response.content)

            def extrair(xpath_expr):
                result = tree.xpath(xpath_expr)
                return result[0].strip() if result else ""

            dados = {
                "Ticker": codigo_indices_BDRs.replace("-", " ").title(),
                "Valor Atual": extrair('//div[@title="Valor (teórico) atual do índice"]//strong[@class="value"]/text()'),
                "Variação": extrair('//div[@title="Variação do valor do índice"]//strong[@class="value"]/text()'),
                "Mín 52 Semanas": extrair('//div[@title="Valor mínimo das últimas 52 semanas"]/strong/text()'),
                "Mín Mês": extrair("//div[@class='d-flex justify-between' and @title='Valor mínimo do mês atual']//span[@class='sub-value']/text()"),
                "Máx 52 Semanas": extrair('//div[@title="Valor máximo das últimas 52 semanas"]/strong/text()'),
                "Máx Mês": extrair('//div[@title="Valor máximo do mês atual"]/span[@class="sub-value"]/text()')  
            }

            dados_extraidos.append(dados)
        else:
            print(f"Erro ao acessar {codigo_indices_BDRs}: {response.status_code}")
    except Exception as e:
        print(f"Erro ao buscar {codigo_indices_BDRs}: {e}")

# Processar os tickers
for indices_BDRs in lista_indices_BDRs:
    extrair_dados_acao(indices_BDRs)
    time.sleep(1)  # Evita sobrecarga

# Converter para DataFrame
df_indices_BDRs = pd.DataFrame(dados_extraidos)

# Exibir os resultados
df_indices_BDRs

# Salvar o DataFrame em CSV
caminho_arquivo = r"C:\Users\AGFAKZZ\Desktop\Status Invest Web Scrapping\df_indice_bdrs.csv"
df_indices_BDRs.to_csv(caminho_arquivo, index=False, encoding='utf-8-sig')

# Índices Internacionais

In [8]:
import requests
from lxml import html
import time
import pandas as pd

pd.set_option('display.max_columns', None)

lista_indices_internacionais = [
    "sp-midcap",
    "sp-100",
    "sp-500"
]

# Lista para armazenar os dados
dados_extraidos = []

def extrair_dados_acao(codigo_indices_internacionais):
    url = f"https://statusinvest.com.br/indices/eua/{codigo_indices_internacionais.lower()}"
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            tree = html.fromstring(response.content)

            def extrair(xpath_expr):
                result = tree.xpath(xpath_expr)
                return result[0].strip() if result else ""

            dados = {
                "Ticker": codigo_indices_internacionais.replace("-", " ").title(),
                "Valor Atual": extrair('//div[@title="Valor (teórico) atual do índice"]//strong[@class="value"]/text()'),
                "Variação": extrair('//div[@title="Variação do valor do índice"]//strong[@class="value"]/text()'),
                "Mín 52 Semanas": extrair('//div[@title="Valor mínimo das últimas 52 semanas"]/strong/text()'),
                "Mín Mês": extrair("//div[@class='d-flex justify-between' and @title='Valor mínimo do mês atual']//span[@class='sub-value']/text()"),
                "Máx 52 Semanas": extrair('//div[@title="Valor máximo das últimas 52 semanas"]/strong/text()'),
                "Máx Mês": extrair('//div[@title="Valor máximo do mês atual"]/span[@class="sub-value"]/text()')  
            }

            dados_extraidos.append(dados)
        else:
            print(f"Erro ao acessar {codigo_indices_internacionais}: {response.status_code}")
    except Exception as e:
        print(f"Erro ao buscar {codigo_indices_internacionais}: {e}")

# Processar os tickers
for indices_internacionais in lista_indices_internacionais:
    extrair_dados_acao(indices_internacionais)
    time.sleep(1)  # Evita sobrecarga

# Converter para DataFrame
df_indices_internacionais = pd.DataFrame(dados_extraidos)

# Exibir os resultados
df_indices_internacionais

# Salvar o DataFrame em CSV
caminho_arquivo = r"C:\Users\AGFAKZZ\Desktop\Status Invest Web Scrapping\df_indices_internacionais.csv"
df_indices_internacionais.to_csv(caminho_arquivo, index=False, encoding='utf-8-sig')